<a href="https://colab.research.google.com/github/chandanareddy-enugala/NLP-SLU/blob/main/Copy_of_NLP_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import string

import seaborn as sns

##### Loading Data

In [ ]:
filePath = "/content/drive/MyDrive/NLP/data/train-v2.tsv"
data = pd.read_csv(filePath, lineterminator='\n',error_bad_lines=False, sep = '\t',header= None)
data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,0,1
0,0,@USER @USER a sicrhau bod mwy o arian poced 'd...
1,1,Parti Dolig da gyda tim swyddfa canolog @USER ...
2,0,@USER yeaah ma fe yn wir. ( oh well.
3,1,@USER hahaha idk. 3am oedd y bws ti?
4,0,@USER dwim yn gal llun ohoni?
...,...,...
78604,1,"@USER @USER diolch yn fawr am ein diweddaru, r..."
78605,1,@USER @USER @USER diolch! Digon o opsiynne felly!
78606,0,mor hungover heddiw 🔫
78607,1,"@USER eidiaaaal, awni dydd llun nesa ia? Y air..."


In [ ]:
data.columns = ["responses", "tweets"] 

In [ ]:
class SentimentAnalysis:
    def __init__(self):
        self.positive_words_count = {}
        self.negative_words_count = {}
        self.positive_words_probability = {}
        self.negative_words_probability = {}

    # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% REMOVE DUPLICATES %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    def removeDuplicates(self, data):
        lengthBefore = len(data)

        data = data.drop_duplicates()
        data = data.reset_index(drop=True)

        lengthAfter = len(data)
        print(f"{lengthBefore - lengthAfter} duplicate rows are removed")
        return data

    def plotDataResponse(self, data):
        sns.countplot(data['responses'])
        print(data['responses'].value_counts())

        # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% CLEANING DATA %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

    # Remove (1) @User (2) {URL}, (3) Emojies, (4) Punctuations, (5) Double spaces
    # Indices 0, 15, 1755
    def remove_user_punc(self, tweet):

        # ------------------------------------------------------------------- (1) Removing ('@User')
        tweet = re.sub('@[\w]*', '', tweet)

        # ------------------------------------------------------------------- (2) Removing ('{URL}')
        tweet = re.sub('{URL}', '', tweet)

        # ------------------------------------------------------------------- (3) Removing (Emojies)
        tweet = re.sub('[^a-zA-Z]', ' ', tweet)

        # ------------------------------------------------------------------- (4) Removing (Puncuations : '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
        for pun in string.punctuation:
            tweet = tweet.replace(pun, "")

        # ------------------------------------------------------------------- (5) Removing (Extra Spaces)
        tweet = tweet.replace("  ", "")

        return tweet

    def cleaningData(self, data):
        data['tweets'] = data['tweets'].apply(self.remove_user_punc)
        print("-> Words starting with @ are removed successfully")
        print("-> Words like {URL} are removed successfully")
        print("-> Words like Emojies are removed successfully")
        print("-> Puncuations are removed successfully")
        print("-> Double spaces are removed successfully")
        return data

    # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% WORDS COUNT %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # 0 - Negative; 1 - Positive
    def tweet_words_count(self, tweets, responses):
        # Loop for each Sentence in Data -----------------------------------
        for i in range(len(tweets)):
            tweet_words = tweets[i].split()
            # Loop for each Word in Sentence -------------------------------
            for word in tweet_words:
                # Adding WORD with NEGATIVE FREQUENCY ----------------------
                if responses[i] == 0:
                    if word in self.negative_words_count.keys():
                        self.negative_words_count[word] += 1
                    else:
                        self.negative_words_count[word] = 1
                # Adding WORD and POSITIVE FREQUENCY ----------------------
                else:
                    if word in self.positive_words_count.keys():
                        self.positive_words_count[word] += 1
                    else:
                        self.positive_words_count[word] = 1
        return ''

    # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% WORD PROBABILITIES %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    def words_probability(self):
        total_tweet_words = set(list(self.positive_words_count.keys()) + list(self.negative_words_count.keys()))
        for key in total_tweet_words:
            # KEY POSITIVE Count ---------------------------------------------
            if key in self.positive_words_count.keys():
                key_positive = self.positive_words_count[key]
            else:
                key_positive = 0

            # KEY NEGATIVE Count ---------------------------------------------
            if key in self.negative_words_count.keys():
                key_negative = self.negative_words_count[key]
            else:
                key_negative = 0

            key_total = key_positive + key_negative
            self.positive_words_probability[key] = key_positive / key_total
            self.negative_words_probability[key] = key_negative / key_total
        return ''

    # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% PREPROCESSING DATA %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    def preprocessData(self, data):
        data = self.removeDuplicates(data)
        data = self.cleaningData(data)
        return data

    # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% TRAINING %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    def train(self, tweets, responses):
        _ = self.tweet_words_count(tweets, responses)
        _ = self.words_probability()
        print("Training is completed successfully")

    # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% EVALUATE TEST DATA %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    def evaluate(self, tweets, responses):
        matched_responses = 0
        results = self.predict(tweets)
        for i in range(len(responses)):
            if responses[i]==results[i]:
                matched_responses += 1
        print(f"-> Accuracy : {(matched_responses/len(responses))*100}%")

    # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% PREDICT %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    def predict(self,  tweets):
        positive_tweet_prob = 1
        negative_tweet_prob = 1
        results = []

        for i in range(len(tweets)):
            tweet_words = tweets[i].split()
            for key in tweet_words:
                if key in self.positive_words_probability.keys():
                    positive_tweet_prob *= self.positive_words_probability[key]

                if key in self.negative_words_probability.keys():
                    negative_tweet_prob *= self.negative_words_probability[key]
            if positive_tweet_prob > negative_tweet_prob:
                results.append(1)
            else:
                results.append(0)
        return results

In [ ]:
SA = SentimentAnalysis()
data = SA.preprocessData(data)

165 duplicate rows are removed
-> Words starting with @ are removed successfully
-> Words like {URL} are removed successfully
-> Words like Emojies are removed successfully
-> Puncuations are removed successfully
-> Double spaces are removed successfully


In [ ]:
SA.train(data['tweets'], data['responses'])

Training is completed successfully


In [ ]:
SA.evaluate(data['tweets'], data['responses'])

Accuracy : 50.087242122549526%


In [ ]:
results = SA.predict(data['tweets'])

In [ ]:
result_df = pd.DataFrame(columns=["actual", "pred"])
result_df["actual"] = data['responses']
result_df["pred"] = results

In [ ]:
(result_df['actual']==result_df['pred']).sum()/len(result_df)

0.5008724212254952